# Integration of distributed heat pumps within Hammarby Sjöstad (Paper III)

In [ ]:
import os
from zerobnl import CoSim

import json
import pandas as pd

You can safely ignore the following error (it will also be in the nodes logs):

RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88

-> [Numpy documentation](https://github.com/numpy/numpy/pull/432)

In [ ]:
sim = CoSim()

# Meta models, environments and wrappers

sim.create_meta_model("MetaROOM", [("setTemp", "Cdeg")], [("Tindoor", "Cdeg"), ("wea.y[1]", "Cdeg"), ("QthTOT", "W")]) # Set, Get
sim.create_environment("EnvROOM", "wrapper_room.py", "Dockerfile_na")

sim.create_meta_model("MetaHSUB", [("HSUB_Qdem_unit", "W")], [("HSUB_Qdem_B", "kW")]) # Set, Get
sim.create_environment("EnvHSUB", "wrapper_hsub.py", "Dockerfile_hsub")

sim.create_meta_model("MetaHP", [("HP_Tr_rad", "Cdeg")], [("HP_COP_real", "-"),("HP_Pel", "kW"),("HP_Qth", "kW")]) # Set, Get
sim.create_environment("EnvHP", "wrapper_hp.py", "Dockerfile_hp")

sim.create_meta_model("MetaTES", [("TES_mdot_in", "kgs")], [("TES_SoC", "-"),("TES_Qstored", "kWh"),("TES_Qmove", "kWh")]) # Set, Get
sim.create_environment("EnvTES", "wrapper_tes.py", "Dockerfile_tes")

sim.create_meta_model("MetaEB", [("EB_signal", "-")], [("EB_Pel", "kW")]) # Set, Get
sim.create_environment("EnvEB", "wrapper_eb.py", "Dockerfile_eb")

sim.create_meta_model("MetaEA", [], [("EA_Pel", "kW")]) # Set, Get
sim.create_environment("EnvEA", "wrapper_ea.py", "Dockerfile_ea")

In [ ]:
# Input parameters
ref_m2 = 48.
df=pd.read_csv('AddFiles\Blocks_data.csv',sep=';')
buildings_ref = list(df)[2:4]

In [ ]:
# Electrical substation
esub_get_attrs = []
cpointsALL = []
for building in buildings_ref: 
    cpoints = df[building][7].split(',') # Spilt the loads
    cpointsALL = cpointsALL + cpoints
    # Create as many outlets as connection points
    esub_get_attrs = esub_get_attrs + [("ESUB_Pel_{}".format(cpoint), "MW") for cpoint in cpoints]
    esub_get_attrs = esub_get_attrs + [("ESUB_Qel_{}".format(cpoint), "MVAr") for cpoint in cpoints]

sim.create_meta_model("MetaESUB", [("ESUB_Pel_hp", "kW"),("ESUB_Pel_eb", "kW"),("ESUB_Pel_ea", "kW")],esub_get_attrs) # Set, Get
sim.create_environment("EnvESUB", "wrapper_esub.py", "Dockerfile_esub")

# Electrical substation
ctrl_set_attrs = [("CTRL_Tapt", "Cdeg"),("CTRL_Qdem", "kW"), ("CTRL_TesSoC", "-")]

for building in buildings_ref: 
    slines = df[building][14].split(',') # Spilt the loads
    # Create as many inlets as lines in the sequence
    ctrl_set_attrs = ctrl_set_attrs + [("CTRL_lineSg_{}".format(sline), "%") for sline in slines]

sim.create_meta_model("MetaCTRL",ctrl_set_attrs, [("CTRL_Tts", "Cdeg"),("CTRL_Tr_hp", "Cdeg"),("CTRL_mdot_tes", "kgs"), ("CTRL_signal_EB", "-"), ("CTRL_signal_DH", "kW"), ("CTRL_Qth_spilled", "kW")]) # Set, Get
sim.create_environment("EnvCTRL", "wrapper_ctrl.py", "Dockerfile_ctrl")

In [ ]:
# Nodes (instances)
for building in buildings_ref:    
    sim.add_node("Ctrl{}".format(building), "MetaCTRL", "EnvCTRL", init_values={"CTRL_Tapt":20.,"CTRL_Qdem": 61.,"CTRL_GridS_trafo":10,"CTRL_TesSoC":-1},parameters={"CTRL_slines":df[building][14].split(','),"CTRL_mdot_hp": float(df[building][0]),"CTRL_Tr_min": float(df[building][1]),"CTRL_Tr_max": float(df[building][2])},local=True)
    sim.add_node("Room{}".format(building), "MetaROOM", "EnvROOM", init_values={"setTemp":20.},parameters={"ROOM_FMUname": df[building][10]},files=[df[building][10],df[building][4],df[building][5],df[building][6]],local=True)
    sim.add_node("Hsub{}".format(building), "MetaHSUB", "EnvHSUB", init_values={"HSUB_Qdem_unit":0.6},parameters={"HSUB_m2B": float(df[building][3]),"m2unit": ref_m2},local=True)
    sim.add_node("Hp{}".format(building), "MetaHP", "EnvHP", init_values={"HP_Tr_rad":35.},parameters={"HP_mdot_hp": float(df[building][0])},local=True)
    sim.add_node("Tes{}".format(building), "MetaTES", "EnvTES", init_values={"TES_mdot_in":0.},parameters={"TES_QstoredMAX": float(df[building][8]),"TES_QstoredMIN": float(df[building][9])},local=True)
    sim.add_node("Eb{}".format(building), "MetaEB", "EnvEB", init_values={"EB_signal":0.},local=True)
    sim.add_node("Esub{}".format(building), "MetaESUB", "EnvESUB", init_values={"ESUB_Pel_hp":0.,"ESUB_Pel_eb":0.},parameters={"ESUB_cpoints":df[building][7].split(','),"ESUB_cpointsALL":cpointsALL},local=True)
    sim.add_node("Ea{}".format(building), "MetaEA", "EnvEA", init_values={},parameters={"EA_m2B": float(df[building][3]),"EA_file": "SK90_eload_m2.csv"}, files=["AddFiles\SK90_eload_m2.csv"],local=True)

In [ ]:
# District heating network node

dh_loads=pd.read_csv('AddFiles\DHN_loads.csv',sep=';')
dh_set_attrs = [(dh_load, "kW") for dh_load in dh_loads['name']]

sim.create_meta_model("MetaDH", dh_set_attrs, [("DH_PE", "kW")]) # Set, Get
sim.create_environment("EnvDH", "wrapper_dh.py", "Dockerfile_dh")

sim.add_node("DHnet", "MetaDH", "EnvDH", files=["AddFiles\DHN_loads.csv"],local=True)

# Electricity distribution grid node
# Import the info about the network parameters that you want to use as set/get attributes
data_power_grid_folder = "AddFiles\PowerGridData"

loads = pd.DataFrame(json.load(open(os.path.join(data_power_grid_folder, 'load.json'))))
loads.index = map(int, loads.index)
loads.head()

trafos = pd.DataFrame(json.load(open(os.path.join(data_power_grid_folder, 'trafo.json'))))
trafos.index = map(int, trafos.index)
trafos.head()

lines = pd.DataFrame(json.load(open(os.path.join(data_power_grid_folder, 'line.json'))))
lines.index = map(int, lines.index)
lines.head()

# Create the meta-models and add the grid node
set_attrs = [("load/{}/p_mw".format(load), "MW") for load in loads.name]
set_attrs += [("load/{}/q_mvar".format(load), "MVAr") for load in loads.name]

get_attrs = [("trafo/{}/loading_percent".format(trafo), "%") for trafo in trafos.name]
get_attrs += [("line/{}/loading_percent".format(line), "%") for line in lines.name]

sim.create_meta_model("MetaGrid", set_attrs, get_attrs)
sim.create_environment("EnvGrid", "wrapper_grid.py", "Dockerfile_grid")

files = [os.path.join(data_power_grid_folder, f) for f in os.listdir(data_power_grid_folder)]
sim.add_node("Grid", "MetaGrid", "EnvGrid", files=files,local=True)

In [ ]:
# Links among nodes

for building in buildings_ref:
    sim.add_link("Ctrl{}".format(building), "CTRL_Tts", "Room{}".format(building), "setTemp") # ok                 
    sim.add_link("Ctrl{}".format(building), "CTRL_Tr_hp", "Hp{}".format(building), "HP_Tr_rad") # ok
    sim.add_link("Ctrl{}".format(building), "CTRL_mdot_tes", "Tes{}".format(building), "TES_mdot_in") # ok
    sim.add_link("Ctrl{}".format(building), "CTRL_signal_EB", "Eb{}".format(building), "EB_signal") # ok
    sim.add_link("Room{}".format(building), "Tindoor", "Ctrl{}".format(building), "CTRL_Tapt") # ok
    sim.add_link("Room{}".format(building), "QthTOT", "Hsub{}".format(building), "HSUB_Qdem_unit") # QthTOT means both sh and dhw
    sim.add_link("Hsub{}".format(building), "HSUB_Qdem_B", "Ctrl{}".format(building), "CTRL_Qdem") # ok    
    sim.add_link("Hp{}".format(building), "HP_Pel", "Esub{}".format(building), "ESUB_Pel_hp")    
    sim.add_link("Tes{}".format(building), "TES_SoC", "Ctrl{}".format(building), "CTRL_TesSoC")
    sim.add_link("Eb{}".format(building), "EB_Pel", "Esub{}".format(building), "ESUB_Pel_eb")
    sim.add_link("Ea{}".format(building), "EA_Pel", "Esub{}".format(building), "ESUB_Pel_ea")

    slines = df[building][14].split(',') # Spilt the loads
    for sline in slines:# Create as many outlets as connection points    
        sim.add_link("Grid", "line/{}/loading_percent".format(sline), "Ctrl{}".format(building), "CTRL_lineSg_{}".format(sline))  
    
    cpoints = df[building][7].split(',') # Spilt the loads
    for cpoint in cpoints:# Create as many outlets as connection points
        sim.add_link("Esub{}".format(building), "ESUB_Pel_{}".format(cpoint), "Grid", "load/{}/p_mw".format(cpoint))   
        sim.add_link("Esub{}".format(building), "ESUB_Qel_{}".format(cpoint), "Grid", "load/{}/q_mvar".format(cpoint)) 

In [ ]:
# Create groups from the simulation sequence. Nodes in the same group run in parallel. 
# A group is defined within the first level of square brackets.

seqCTRL = ["Ctrl{}".format(building) for building in buildings_ref]
seqROOM = ["Room{}".format(building) for building in buildings_ref]
seqHSUB = ["Hsub{}".format(building) for building in buildings_ref]
seqHP = ["Hp{}".format(building) for building in buildings_ref]
seqTES = ["Tes{}".format(building) for building in buildings_ref]
seqEB = ["Eb{}".format(building) for building in buildings_ref]
seqESUB = ["Esub{}".format(building) for building in buildings_ref]
seqEA = ["Ea{}".format(building) for building in buildings_ref]

sim.create_sequence([seqCTRL,seqROOM,seqHSUB,seqHP,seqTES,seqEB,["DHnet"],seqEA,seqESUB,["Grid"]])

sim.set_time_unit("seconds")
sim.create_steps([1800.] * 2 * 24 * 1) # sim.create_steps([1800.] * 2 * 24 * 80)

Once the next step has been launched, logging `INFO :: Waiting for local nodes to run..`, you need to run tho following command `wrapper_eplus.py Base1 GRP1` in the indicated folder (in a dedicated environment) in order to run the local node.

In [ ]:
sim.run()

If you see `INFO :: Simulation finished in X min and Y sec` it means everything went well.
You can find logs of the nodes in the file `nodes.log`, it's a text file you can open it directly in Jupyter or in your favorite text editor.

At the begining of the file you will find a serie of:

`Step X/10 : DO SOMETHING
 ---> 29d2f3226daf`
 
It's the logs of the creation of the Docker image, based on the provided Dockerfile (here `Dockerfile_base`).

Then all the logs are structures in the same way:

`<node>    | <level> :: <message>`

* `node` refers to the concerned simulation node or orchestrator
* `level` can be `DEBUG`: used for development purpose, `INFO`: giving you info on the running process, `WARNING`: warning you on action to make or some weird behaviour, `ERROR`: something went wrong and `CRITICAL`: something went really wrong.
* `message` is the body of the log, it describes what's happening.

You can also find information on the ongoing simulation in the file `activity.log` (in the root folder for the main processus and on the temporary folder for each node)

In [ ]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

The name to the stored results are build as `<type>||<node>||<attribute>`.

`type` can be:
* `IN` if it's an input attribute (to set - stored automatically)
* `OUT` if it's an output attribute (to get - stored automatically)
* `X` if it's an internal value (stored by the user, using the `save_attribute()` method in the wrapper)

Knowing this, you can create matching pattern using `*` in order to properly select results.

In [ ]:
import matplotlib.pyplot as plt
start = 1+(24 * 2) * 0 # (day) * day number
stop = (24 * 2) * 80
jump = 2

In [ ]:
room = sim.get_results_by_pattern("OUT*RoomB1*")
room.keys()
plt.figure(figsize=(18, 8))
ro = room['OUT||RoomB1||Tindoor']
time_plot =list(range(int(len(ro)/2)))
plt.plot(time_plot,ro[start:stop:jump]-273, "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Tindoor (°C)',fontsize=20)

In [ ]:
plt.figure(figsize=(18, 8))
ro = room['OUT||RoomB1||QthTOT']
plt.plot(time_plot,ro[start:stop:jump]/1000, "o-", alpha=1)
plt.ylim(bottom=0)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Qdot_heat (kW)',fontsize=20)
plt.savefig("Results\Figures\Qheat_B5.png")

In [ ]:
plt.figure(figsize=(18, 8))
ro = room['OUT||RoomB1||wea.y[1]']
plt.plot(time_plot,ro[start:stop:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Toutdoor (°C)',fontsize=20)
plt.savefig("Results\Figures\Toutdoor.png")

In [ ]:
plt.figure(figsize=(18, 8))
sub = sim.get_results_by_pattern("OUT*HsubB1*")
su = sub['OUT||HsubB1||HSUB_Qdem_B']
plt.plot(time_plot,su[start:stop:jump], "o-", alpha=1)
plt.ylim(bottom=0)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Qdot_heat (kW)',fontsize=20)
plt.savefig("Results\Figures\Qheat_B5block.png")

In [ ]:
start_z = 1+(24 * 2) * 8 # (day) * day number
stop_z = (24 * 2) * 9
plt.figure(figsize=(18, 8))
control = sim.get_results_by_pattern("OUT*CtrlB5*")
co = control['OUT||CtrlB5||CTRL_Tr_hp'] 
plt.plot(co[start_z:stop_z:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Tret to HP (°C)',fontsize=20)
plt.savefig("Results\Figures\HP_Tret.png")

In [ ]:
plt.figure(figsize=(18, 8))
co = control['OUT||CtrlB5||CTRL_mdot_tes'] # 
plt.plot(time_plot,co[start:stop:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Mdot to TES (kg/s)',fontsize=20)
plt.savefig("Results\Figures\TES_Mdot.png")

In [ ]:
plt.figure(figsize=(18, 8))
heatpump = sim.get_results_by_pattern("OUT*HpB5*")
he = heatpump['OUT||HpB5||HP_Pel'] # 
plt.plot(time_plot,he[start:stop:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Electrical power HP (kW)',fontsize=20)
plt.savefig("Results\Figures\HP_Pel.png")

In [ ]:
plt.figure(figsize=(18, 8))
heatpump2 = sim.get_results_by_pattern("OUT*EsubB5*")
he2 = heatpump2['OUT||EsubB5||ESUB_Pel_Load_R'] # 
plt.plot(time_plot,he2[start:stop:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Electrical power substation (MW)',fontsize=20)
plt.savefig("Results\Figures\ESUB_Pel.png")

In [ ]:
plt.figure(figsize=(18, 8))
thermalstorage = sim.get_results_by_pattern("OUT*TesB5*")
ts = thermalstorage['OUT||TesB5||TES_SoC'] # 
plt.plot(time_plot,ts[start:stop:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='TES State of charge (-)',fontsize=20)
plt.savefig("Results\Figures\TES_SoC.png")

In [ ]:
plt.figure(figsize=(18, 8))
grid = sim.get_results_by_pattern("OUT*Grid*")
gr = grid['OUT||Grid||trafo/trafo_SK90/loading_percent'] # 
plt.plot(time_plot,gr[start:stop:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='trafo_SK90 loading (%)',fontsize=20)
plt.savefig("Results\Figures\Trafo_loading.png")

In [ ]:
start_z = 1+(24 * 2) * 9 # (day) * day number
stop_z = (24 * 2) * 10
plt.figure(figsize=(18, 8))
plt.plot(gr[start_z:stop_z:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='trafo_SK90 loading (%)',fontsize=20)
plt.savefig("Results\Figures\Trafo_loading_1d_ok.png")

In [ ]:
start_z = 0+(24 * 2) * 9 # (day) * day number
stop_z = (24 * 2) * 10
plt.figure(figsize=(18, 8))
plt.plot(gr[start_z:stop_z:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='trafo_SK90 loading (%)',fontsize=20)
plt.savefig("Results\Figures\Trafo_loading_1d_NOTok.png")

In [ ]:
#plt.figure(figsize=(18, 8))
fig, ax = plt.subplots()
grid = sim.get_results_by_pattern("OUT*Grid*")
[ax.plot(time_plot,grid['OUT||Grid||line/{}/loading_percent'.format(ln)][start:stop:jump], "o-", alpha=1,label=ln) for ln in lines.name]
ax.legend()
fig.set_figheight(8)
fig.set_figwidth(18)

plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='lines loading (%)',fontsize=20)
plt.savefig("Results\Figures\Lines_loading.png")

In [ ]:
plt.figure(figsize=(18, 8))
elback = sim.get_results_by_pattern("OUT*EaB5*")
eb = elback['OUT||EaB5||EA_Pel'] # 
plt.plot(time_plot,eb[start:stop:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Electrical appliances (kW)',fontsize=20)

In [ ]:
plt.figure(figsize=(18, 8))
elback = sim.get_results_by_pattern("OUT*EbB5*")
eb = elback['OUT||EbB5||EB_Pel'] # 
plt.plot(time_plot,eb[start:stop:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Electrical power back up (kW)',fontsize=20)

In [ ]:
plt.figure(figsize=(18, 8))
control2 = sim.get_results_by_pattern("OUT*CtrlB16*")
DH_PE = control2['OUT||CtrlB16||CTRL_signal_DH'] # 
plt.plot(time_plot,DH_PE[start:stop:jump], "o-", alpha=1)
plt.xlabel(xlabel='time (h)',fontsize=20)
plt.ylabel(ylabel='Primary energy DH (kW)',fontsize=20)

In [ ]:
# Save to csv for TE analysis

# For CR a single call is enough
efCR = pd.DataFrame({'DH_PE_MW':DH_PE})
efCR.to_csv('Results\Files\EnergyFlows.csv')

# For DR a loop is required
efDR = pd.DataFrame()

for building in buildings_ref:
    
    # Collect outputs to save
    heatpump = sim.get_results_by_pattern("OUT*Hp{}*".format(building))    
    efDR['HP_Pel_kW'] = heatpump['OUT||Hp{}||HP_Pel'.format(building)]
    efDR['HP_Qth_kW'] = heatpump['OUT||Hp{}||HP_Qth'.format(building)]
    
    thermalstorage = sim.get_results_by_pattern("OUT*Tes{}*".format(building))
    efDR['TES_Qflow_kWh'] = thermalstorage['OUT||Tes{}||TES_Qmove'.format(building)]
    efDR['TES_SoC'] = thermalstorage['OUT||Tes{}||TES_SoC'.format(building)]
    
    elback = sim.get_results_by_pattern("OUT*Eb{}*".format(building))
    efDR['EB_Pel_kW'] = elback['OUT||Eb{}||EB_Pel'.format(building)]
    
    heatSub = sim.get_results_by_pattern("OUT*Hsub{}*".format(building))
    efDR['HSUB_Qdem_B_kW'] = heatSub['OUT||Hsub{}||HSUB_Qdem_B'.format(building)]   
    
    room = sim.get_results_by_pattern("OUT*Room{}*".format(building))
    efDR['ROOM_Tindoor_Cdeg'] = room['OUT||Room{}||Tindoor'.format(building)]   
    
    control = sim.get_results_by_pattern("OUT*Ctrl{}*".format(building))
    efDR['CTRL_Tth_Cdeg'] = control['OUT||Ctrl{}||CTRL_Tts'.format(building)] 
    efDR['CTRL_Qth_spilled_kW'] = control['OUT||Ctrl{}||CTRL_Qth_spilled'.format(building)]  
    efDR['CTRL_QDH_B_kW'] = control['OUT||Ctrl{}||CTRL_signal_DH'.format(building)] 

    # Save to csv
    path = "Results\Files\{}\EnergyFlows.csv".format(building)
        
    os.makedirs(os.path.dirname(path), exist_ok=True) # create directory and file
    
    efDR.to_csv('Results\Files\{}\EnergyFlows.csv'.format(building))
    
# For the loadings on the cables and the transformer a different loop is required
efEDG = pd.DataFrame()
grid = sim.get_results_by_pattern("OUT*Grid*")

efEDG['trafo_SK90'] = grid['OUT||Grid||trafo/trafo_SK90/loading_percent']
for ln in lines.name:    
    # Collect outputs to save  
    efEDG[ln] = grid['OUT||Grid||line/{}/loading_percent'.format(ln)]

path = "Results\Files\GridLoading.csv"      
os.makedirs(os.path.dirname(path), exist_ok=True) # create directory and file    
efEDG.to_csv('Results\Files\GridLoading.csv')